In [ ]:
import os

import django
from django.core.exceptions import ObjectDoesNotExist
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt
from random import randint

# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependency of django_ledger...

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_ledger_starter.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
os.chdir('../')

django.setup()

from django_ledger.models import EntityModel, ChartOfAccountModel
from django_ledger import __version__ as DJL_VERSION
from django.contrib.auth import get_user_model

print(f'Using Django Ledger Version v{DJL_VERSION}')

# CoA Account
- A Chart of Accounts is an index of accounts used to transact on the General Ledger of an Entity. 
- It provides a human-readable and digestible breakdown of all companies assets, liabilities and equity accounts.
- The Entity Chart of Accounts is responsible for driving a lot of the Financial Reporting details. 
- A healthy breakdown of different accounts and sub-accounts will provide a clearer view and understanding of the company's financial health.
- Without a CoA the EntityModel cannot transact since no accounts are present.

 <img src="./assets/djl_core_model_coa.png" alt="Django Ledger Core Model" width="1200" height="600"> 

# Get or Create Your Entity Administrator UserModel

In [ ]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    ceo_user = UserModel.objects.get(username__exact=MY_USERNAME)
except:
    ceo_user = UserModel(username=MY_USERNAME)
    ceo_user.set_password(MY_PASSWORD)
    ceo_user.save()
    
ceo_user

# Get or Create an Entity Model

In [ ]:
ENTITY_NAME = 'Chart of Accounts Notebook, LLC'

try:
    entity_model = EntityModel.objects.for_user(user_model=ceo_user).get(name__exact=ENTITY_NAME)
except ObjectDoesNotExist:
    entity_model = EntityModel.create_entity(
        name=ENTITY_NAME,
        admin=ceo_user,
        use_accrual_method=False,
        fy_start_month=1
    )
    
entity_model

# List the ChartofAccounts models

In [ ]:
# newly created entities do not have a default CoA...
entity_model.has_default_coa()

In [ ]:
entity_model.chartofaccountmodel_set.all()

In [ ]:
URL = entity_model.get_coa_list_url()
URL = f'http://localhost:8000{URL}'
URL

# Create A Chart of Accounts

In [ ]:
coa_model = entity_model.create_chart_of_accounts(
    coa_name=f'My First Chart of Accounts #{randint(10000,99999)}',
    commit=True
)

In [ ]:
coa_model

## Create a Chart of Accounts
- Newly created EntityModel do not have a default Code of Accounts yet.

In [ ]:
entity_model.has_default_coa()

In [ ]:
a_coa_model = entity_model.create_chart_of_accounts(
    commit=True,
    coa_name='My QuickStart CoA'
)

In [ ]:
coa_model_qs = entity_model.chartofaccountmodel_set.all()
pd.DataFrame(coa_model_qs.values())

# Assign a CoA as Default

In [ ]:
entity_model.set_default_coa(coa_model=a_coa_model, commit=True)

In [ ]:
entity_model.has_default_coa()

In [ ]:
entity_model.default_coa

In [ ]:
entity_model.default_coa == a_coa_model

## Can be set also by slug

In [ ]:
entity_model.set_default_coa(coa_model=a_coa_model.slug, commit=True)

In [ ]:
entity_model.default_coa == a_coa_model

## If using Entity Model API, Provided CoA Model or Slug is Validated

In [ ]:
entity_model.set_default_coa(coa_model=ChartOfAccountModel(slug='this-is-not-for-this-entity'), commit=True)

In [ ]:
entity_model.set_default_coa(coa_model='this-is-not-for-this-entity', commit=True)

# Chart of Account Root Accounts - A Hierachy of Accounts

In [ ]:
coa_root_accounts_qs = coa_model.accountmodel_set.all().is_coa_root().select_related('coa_model').order_by('code')
pd.DataFrame(coa_root_accounts_qs.values('code', 'name', 'coa_model__name', 'depth', 'role', 'locked'))

In [ ]:
coa_tree = coa_model.get_coa_account_tree()
G = nx.from_dict_of_dicts({
    f"{l1['data']['code']}\n{l1['data']['name']}\nLocked: {l1['data']['locked']}": {
        f"{l2['data']['code']}\n{l2['data']['name']}\nLocked: {l2['data']['locked']}": {
            'weight': 1
        } for l2 in l1['children']
    } for l1 in coa_tree
}, create_using=nx.DiGraph)

options = {
    'font_size': 14,
    # 'font_color': 'red',
    'node_size': 8000,
    'node_color': 'white',
    'edgecolors': 'black',
    'linewidths': 2,
    'width': 3,
}

fig, ax = plt.subplots(figsize=(12, 12))
ax.margins(0.2, 0.2)
nx.draw_networkx(G, pos=nx.planar_layout(G), **options)
ax.set_title(f'Django Ledger | Initial Code of Accounts Structure')

# Active Status

In [ ]:
coa_model.is_active()

In [ ]:
coa_model.mark_as_active()

In [ ]:
coa_model.mark_as_active(raise_exception=True)

In [ ]:
coa_model.mark_as_inactive(commit=True)

In [ ]:
coa_model.is_active()